### HuggingFace gguf 파일(언어 모델 파일) 을 ollama 를 이용해 가져오기
- GGUF 파일이란?<br>
모델을 빠르게 로딩할 수 있는 2024년3월1일 기준 최신 모델 형식<br>
호환성이 좋으며, 키-값 구조를 사용합니다.<br>
<a href="https://github.com/ggerganov/ggml/blob/master/docs/gguf.md">GGUF file format</a>

#### 작업 순서
1. HuggingFace 레포지토리 가져오기
2. gguf 파일 가져오기
3. 로컬 저장 경로 설정하기

#### 기본 언어 모델 - 다른 모델로 해도 됩니다.
- 베이스 모델(2024.3월28일자_기준) : https://huggingface.co/upstage/SOLAR-10.7B-v1.0
- 야놀자 데이터 학습 모델(2024.3.28일자_기준) : https://huggingface.co/yanolja/EEVE-Korean-Instruct-10.8B-v1.0
- 야놀자 데이터 gguf 변환 모델(사용할 파일) : https://huggingface.co/heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF
- 직접 다운로드(선택해서 다운로드 받기) : https://huggingface.co/heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF/tree/main


In [1]:
!pip install huggingface-hub

### 1. 모델 다운로드

In [4]:
from huggingface_hub import hf_hub_download

model_name_or_path = "heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF"
model_basename = "ggml-model-Q4_K_M.gguf" # 파일 선택 - gguf 변환모델 링크 들어가서 file 확인
save_dir = "./models"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename, local_dir=save_dir)
print(model_path)

./models\ggml-model-Q4_K_M.gguf


# 현재 모델의 저장 경로를 복사 해주세요
예시 : D:\Projects\LLM_Vision_Projects\Part1_LLM\models<br>
터미널 창에서 cd D:\Projects\LLM_Vision_Projects\Part1_LLM\models

In [1]:
!pip install ollama

  Obtaining dependency information for ollama from https://files.pythonhosted.org/packages/9a/55/87a9a2db552f787b68c8bfd720371567c13512023147b166547bfe6f9331/ollama-0.1.8-py3-none-any.whl.metadata


### 1. ollama를 이용해서 새로 다운로드한 로컬 모델 실행해보기

In [3]:
import ollama

stream = ollama.chat(
    model='EEVE-Korean-10.8B-Q4_K_M',
    messages=[{'role': 'user', 'content': '오늘 날씨가 어때요?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

현재 제가 AI 언어 모델로서 실시간 날씨 정보를 제공할 수 있는 능력은 없지만, 현재 위치의 기상 조건을 알아보는 데 도움을 드릴 수 있습니다.

구글이나 빙과 같은 검색 엔진을 사용하여 "날씨"라고 입력하고 귀하의 도시나 지역을 함께 검색해보세요. 이렇게 하면 현재 기온, 습도, 그리고 예보에 대한 정보를 찾을 수 있을 것입니다.

또는 날씨 앱을 사용하거나 스마트폰에서 날씨 위젯을 확인하여 기상 조건을 확인할 수도 있습니다.

기억하세요, 날씨가 항상 변동될 수 있으니 최신 정보를 얻기 위해 정기적으로 업데이트를 확인하는 것이 중요합니다.

### 2. 랭체인에 연결해보기

In [4]:
for models in ollama.list()['models']:
    print(models['model'])

EEVE-Korean-10.8B-Q4_K_M:latest
llama2:latest


In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(model="EEVE-Korean-10.8B-Q4_K_M:latest")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 데이브 더 다이브의 게임 가이드 입니다. 100자 내외로 한글로 답변하세요"
        ),
        
        MessagesPlaceholder(variable_name="history"), # history 이름의 메세지 저장 객체
        (
            "human",
            "{input}" # 사용자 입력을 변수로 받기
        )
    ]
)
chain = prompt | model | StrOutputParser()

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_store = {} # 메시지 기록(세션)을 저장할 딕셔너리

In [7]:
# 세션 ID 를 기반으로 세션기록 가져오는 함수 만들기
def get_session_history(session_ids : str) -> BaseChatMessageHistory:
    print(session_ids)

    if session_ids not in session_store: # 세션 기록이 없을 경우
        # 새로운 객체 생성해서 세션 스토어에 저장하기
        session_store[session_ids] = ChatMessageHistory()
    return session_store[session_ids] # 저장된 세션 ID 기록 반환하기

In [8]:
with_message_history = (
    RunnableWithMessageHistory(
        runnable=chain,
        get_session_history=get_session_history,
        input_messages_key="input",
        history_messages_key='history',
    )
)

In [9]:
for token in with_message_history.stream(
    {"input": "무기에 대해 알려주세요"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},):
    print(token, end="")

abc123
게임에서 무기는 캐릭터가 적과 전투를 벌이고, 장애물을 극복하며, 목표를 달성하는 데 사용하는 도구입니다. 다양한 종류의 무기들이 있으며 각각 고유한 능력, 공격 범위, 속도 및 재사용 대기시간이 있습니다. 몇 가지 예로는 검, 도끼, 활, 총기류 등이 있습니다.

검은 근접전에 특화된 무기로 빠른 공격 속도를 가지고 있고 적에게 가까운 거리에서 피해를 입힙니다. 도끼는 더 큰 피해량을 가지고 있지만 느린 공격 속도를 가지고 있으며, 주로 중거리 전투에 적합합니다. 활과 총기는 원거리 무기로서 멀리서 적을 공격할 수 있습니다.

무기를 업그레이드하거나 새로운 무기를 잠금 해제하여 캐릭터의 능력을 향상시킬 수 있습니다. 또한 다양한 종류의 갑옷도 있어 방어력을 강화하고 특정 능력이나 효과를 부여받을 수 있습니다.

In [10]:
for token in with_message_history.stream(
    {"input": "아까 질문한 내용이 뭐였죠?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},):
    print(token, end="")

abc123
당신의 이전 질문에 대한 답변은 다음과 같습니다: "무기에 대해 알려주세요." 

게임에서 무기는 캐릭터가 적과 전투를 벌이고, 장애물을 극복하며, 목표를 달성하는 데 사용하는 도구입니다. 다양한 종류의 무기들이 있으며 각각 고유한 능력, 공격 범위, 속도 및 재사용 대기시간이 있습니다.

검은 근접전에 특화된 무기로서 빠른 공격 속도를 가지고 있고 적에게 가까운 거리에서 피해를 입힙니다. 도끼는 더 큰 피해량을 가지고 있지만 느린 공격 속도를 가지고 있으며 주로 중거리 전투에 적합합니다. 활과 총기는 원거리 무기로서 멀리서 적을 공격할 수 있습니다.

무기를 업그레이드하거나 새로운 무기를 잠금 해제하여 캐릭터의 능력을 향상시킬 수 있습니다. 또한 다양한 종류의 갑옷도 있어 방어력을 강화하고 특정 능력이나 효과를 부여받을 수 있습니다.

### 3. RAG(자료 검색 기능) + OLLAMA(로컬모델)

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 저장해놓은 vector DB 불러오기
dive_db = Chroma(persist_directory="./chroma_db",
             embedding_function=OpenAIEmbeddings())

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

model = ChatOllama(model="EEVE-Korean-10.8B-Q4_K_M:latest")

# 질문 양식 설정 - 템플릿 설정
template="""당신은 데이브 더 다이브의 게임 가이드 입니다. {context} 를 기반으로 qustion에 100자 내외로 한글로 답변하세요 
Question :{question}
"""

prompt = ChatPromptTemplate.from_template(template=template)
retriever = dive_db.as_retriever(search_kwargs={"k":5})

parser = StrOutputParser()

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join([doc.page_content for doc in docs])


# 단계 RAG 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

In [46]:
query = "무기에 대해서 알려줘"
response = rag_chain.invoke(query)
print(response)

데이브 더 다이버의 무기들은 근접 무기와 작살, 총기로 분류되며, 각기 독특한 속성을 가지고 있습니다. 화염, 전기, 수면, 독, 마비, 빙결과 같은 속성들이 있으며 각각 고유한 효과를 가집니다.

근접무기는 수중 나이프로 구성되어 있고, 작살은 다양한 속성이 있는 여러 종류가 있습니다. 예를 들어, 수면 무기들은 어류를 재워 회수할 수 있게 해주고, 전기 작살은 주변 물고기에게 감전을 일으킵니다. 마비와 빙결 속성을 가진 작살도 있으며, 이는 어류의 행동을 제어하는 데 도움이 됩니다.

총기는 그물탄을 발사하여 동시 포획이 가능한 최대 7마리의 어류를 잡을 수 있습니다. 소형과 중형 네트건 두 종류가 있으며, 각각 독특한 사거리와 포획 능력을 가지고 있습니다.

무기의 등급은 어류 사냥 시 사용 방법에 따라 달라집니다. 수면 무기나 그물로 잡은 물고기는 더 높은 등급으로 획득할 수 있고, 그렇지 않으면 낮은 등급을 받게 됩니다.

업그레이드 키트를 사용하여 무기를 강화하면 동시 포획 가능한 최대 어류 수가 늘어나거나 속성이 향상될 수 있습니다.
